In [1]:
import pandas as pd
import datetime
import warnings
warnings.filterwarnings("ignore")

# puxar arquivo do banco
df = pd.read_csv('eapv_29032022_10h28.csv', sep=';', encoding='utf-8')

print(f'Banco importado: {df.shape}')
banco_completo = df.shape[0]

Banco importado: (18350, 189)


# Excluir as fichas canceladas


In [2]:
# Transformando ['Número da Notificação'] para string
df['Número da Notificação'] = df['Número da Notificação'].astype(str)
# puxar arquivo canceladas
df_canceladas = pd.read_csv('eapv_canceladas_29032022.csv', sep=';', encoding='utf-8')
# df_canceladas passa a ser apenas a coluna 'Número da Notificação' e em string
df_canceladas = df_canceladas[['Número da Notificação']].astype(str)
print(f'Banco canceladas: {df_canceladas.shape}')
banco_canceladas = df_canceladas.shape[0]
# Transformando o banco df_canceladas para uma lista notif_canceladas
notif_canceladas = df_canceladas['Número da Notificação'].to_list()

# TIRANDO AS EXCLUÍDAS DO BANCO DE DADOS 
df = df[~df['Número da Notificação'].str.contains('(?i)' + '|'.join(notif_canceladas), na=False)]

# conferir
print(f'Banco sem as canceladas: {df.shape[0]}')
print(f'Banco completo: {banco_completo} - canceladas: {banco_canceladas} = {banco_completo - banco_canceladas}')

# Verificar se ficou alguma cancelada e excluir do banco
df[df['Situação da Notificação'] == 'Cancelado']
df = df[~df['Situação da Notificação'].str.contains('Cancelado', na=False)]

Banco canceladas: (1012, 1)
Banco sem as canceladas: 17338
Banco completo: 18350 - canceladas: 1012 = 17338


# Tratar vacinas

In [3]:
# Manter no df apenas vacinas corretas 
df = df.loc[(df['Imunobiológico (vacina)'].str.contains('(?i)Butantan') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Sinovac') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Oxford') |
             df['Imunobiológico (vacina)'].str.contains('(?i)AstraZeneca') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Covishield') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Pfizer') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Janssen'))]

bd_vacinas_covid = df.shape[0]

print(f'Foram excluídas do banco {banco_completo - bd_vacinas_covid} fichas que contém outras vacinas que não as COVID-19')
print(f'Banco somente com vacinas COVID-19: {df.shape}')

Foram excluídas do banco 1053 fichas que contém outras vacinas que não as COVID-19
Banco somente com vacinas COVID-19: (17297, 189)


.

.

# ANÁLISE

In [90]:
# GRAVES E NAO GRAVES ok
# FAIXA ETARIA ok
# SEMANA EPIDEMIOLOGICA ok
# ERRO DE IMUNIZAÇÃO OU EVENTO ADVERSO ok
# ENCERRADOS E NÃO ENCERRADOS ok

In [91]:
## DASHBOARD ENCERRAMENTOS ##
# ORGÃO DE ENCERRAMENTO (VER A QUESTÃO DE DUPLICADAS) ok
# VER OS MUNICIPIOS  DOS ENCERRAMENTOS MUNICIPAIS

In [12]:
df_especial = df.copy()

GRAVIDADE

In [93]:
# grave e não grave
df_especial['Gravidade'] = 'Nao grave'
df_especial.loc[df_especial['Classificação de gravidade'].str.contains('Grave', na = False), 'Gravidade'] = 'Grave'

TIPO DE EVENTO

In [94]:
# Tipo de evento
df_eventos = df.copy()
df_eventos_separado = pd.DataFrame()

# fazendo um split na coluna e contatenando as informações
for index, row in df_eventos.iterrows():
    eventos = row['Tipo de Evento'].split('| ')
        
    i = 0
    dados = {}
    for evento in eventos:
        # Trata os dados e concatena depois no dataframe
        dados.setdefault("Número da Notificação", []).append(row['Número da Notificação'])
        dados.setdefault("Tipo de Evento", []).append(evento)

        i += 1

    df_eventos_separado = pd.concat([df_eventos_separado, pd.DataFrame(dados)])
df_eventos_separado

# criando novas colunas no df separado
df_eventos_separado['Evento'] = pd.Series()
df_eventos_separado['Erro'] = pd.Series()

# pesquisa no df separado e colocando sim nos que True
df_eventos_separado.loc[df_eventos_separado['Tipo de Evento'].str.contains('Evento Adverso', na = False), 'Evento'] = 'Sim'
df_eventos_separado.loc[df_eventos_separado['Tipo de Evento'].str.contains('Erro de Imunização', na = False), 'Erro'] = 'Sim'

# Agrupando por numero de notificação volta o N de notificadas
df_eventos_separado = df_eventos_separado.groupby(by=['Número da Notificação']).count()

# Tipo de evento começa com ambos os tipos
df_eventos_separado['Tipo de Evento'] = 'Evento adverso e Erro de imunização'
# Se evento for igual a 0 e diferente de 0 = Erro
df_eventos_separado.loc[(df_eventos_separado['Evento'] == 0) & (df_eventos_separado['Erro'] != 0), 'Tipo de Evento'] = 'Erro de imunização'
# Se evento for diferente de 0 e igual a 0 = Evento
df_eventos_separado.loc[(df_eventos_separado['Evento'] != 0) & (df_eventos_separado['Erro'] == 0), 'Tipo de Evento'] = 'Evento adverso'
# Apagar colunas 
df_eventos_separado = df_eventos_separado.drop(columns=['Evento'])
df_eventos_separado = df_eventos_separado.drop(columns=['Erro'])

# Merge com a coluna df_especial
df_especial = df_especial.merge(df_eventos_separado, on='Número da Notificação', how='left')
# renomeando coluna
df_especial = df_especial.rename(columns={'Tipo de Evento_y':'Tipo de Evento'})

SEMANA EPIDEMIOLOGICA

In [95]:
# semana epidemiologica

# transformando em datetime
df_especial['Data da Notificação'] = pd.to_datetime(df_especial['Data da Notificação'], infer_datetime_format=True, format='%d/%m/%Y')
df_especial['SE'] = pd.Series()

# separando df de 2021 e 2022
# Insert Epidemiology Week number
df_especial.loc[(df_especial['Data da Notificação'] < '2022-01-01'), 'SE'] = df_especial['Data da Notificação'].dt.strftime('2021_' + '%U')
df_especial.loc[(df_especial['Data da Notificação'] >= '2022-01-01'), 'SE'] = df_especial['Data da Notificação'].dt.strftime('2022_' + '%U')

# tratando o SE_00 e 2021 < 03
df_especial.loc[df_especial['SE'] == '2022_00', 'SE'] = '2021_52'
df_especial.loc[df_especial['SE'] < '2021_03', 'SE'] = pd.Series()

df_especial['SE'].unique()

array(['2021_52', '2021_12', '2021_51', '2021_41', '2021_38', '2021_47',
       '2021_49', '2021_45', '2021_50', '2022_01', '2021_42', '2021_39',
       '2021_40', '2022_02', '2021_48', '2022_03', '2021_46', '2022_04',
       '2021_43', '2022_05', '2022_06', '2021_30', '2022_07', '2021_34',
       '2022_08', '2022_09', '2022_10', '2022_11', '2021_10', '2021_20',
       '2022_12', '2021_37', '2022_13', '2021_24', '2021_25', '2021_26',
       '2021_27', '2021_28', '2021_29', '2021_31', '2021_35', '2021_03',
       '2021_08', '2021_36', '2021_32', '2021_09', '2021_05', '2021_18',
       '2021_06', '2021_16', '2021_19', '2021_44', '2021_17', '2021_15',
       '2021_04', '2021_21', '2021_33', '2021_23', '2021_14', '2021_22',
       '2021_07', '2021_13', '2021_11', nan], dtype=object)

FAIXA ETÁRIA

In [96]:
# faixa etaria
df_especial['Faixa etaria'] = 'Indefinido'
df_especial.loc[(df_especial['Idade Evento'] >= 5) & (df_especial['Idade Evento'] <= 11), 'Faixa etaria'] = '5-11'
df_especial.loc[(df_especial['Idade Evento'] >= 12) & (df_especial['Idade Evento'] <= 19), 'Faixa etaria'] = '12-19'
df_especial.loc[(df_especial['Idade Evento'] >= 20) & (df_especial['Idade Evento'] <= 29), 'Faixa etaria'] = '20-29'
df_especial.loc[(df_especial['Idade Evento'] >= 30) & (df_especial['Idade Evento'] <= 39), 'Faixa etaria'] = '30-39'
df_especial.loc[(df_especial['Idade Evento'] >= 40) & (df_especial['Idade Evento'] <= 49), 'Faixa etaria'] = '40-49'
df_especial.loc[(df_especial['Idade Evento'] >= 50) & (df_especial['Idade Evento'] <= 59), 'Faixa etaria'] = '50-59'
df_especial.loc[(df_especial['Idade Evento'] >= 60) & (df_especial['Idade Evento'] <= 69), 'Faixa etaria'] = '60-69'
df_especial.loc[(df_especial['Idade Evento'] >= 70) & (df_especial['Idade Evento'] <= 79), 'Faixa etaria'] = '70-79'
df_especial.loc[df_especial['Idade Evento'] >= 80, 'Faixa etaria'] = '80+'

ENCERRADOS

In [5]:
# Encerrados
df_especial['Encerramento'] = 'Nao encerrado'

# Municipio
df_especial['Municipal'] = 'Municipal'
df_especial['Encerramento Municipal - Evento adverso'].fillna('Not', inplace=True)
df_especial.loc[df_especial['Encerramento Municipal - Evento adverso'].str.contains('Not', na = False), 'Municipal'] = 'Nao encerrado'
# Estadual
df_especial['Estadual'] = 'Estadual'
df_especial['Encerramento Estadual - Evento adverso'].fillna('Not estadual', inplace=True)
df_especial.loc[df_especial['Encerramento Estadual - Evento adverso'].str.contains('Not estadual', na = False), 'Estadual'] = 'Nao encerrado'
# Federal 
df_especial['Federal'] = 'Federal'
df_especial['Encerramento Federal - Evento adverso'].fillna('Not federal', inplace=True)
df_especial.loc[df_especial['Encerramento Federal - Evento adverso'].str.contains('Not federal', na = False), 'Federal'] = 'Nao encerrado'
# Cifavi 
df_especial['Cifavi'] = 'Cifavi'
df_especial['Encerramento Cifavi - Evento adverso'].fillna('Not cifavi', inplace=True)
df_especial.loc[df_especial['Encerramento Cifavi - Evento adverso'].str.contains('Not cifavi', na = False), 'Cifavi'] = 'Nao encerrado'

# Agrupando informações na coluna Encerramento
df_especial.loc[df_especial['Municipal'].str.contains('Municipal', na = False), 'Encerramento'] = 'Municipal'
df_especial.loc[df_especial['Estadual'].str.contains('Estadual', na = False), 'Encerramento'] = 'Estadual'
df_especial.loc[df_especial['Federal'].str.contains('Federal', na = False), 'Encerramento'] = 'Federal'
df_especial.loc[(df_especial['Municipal'] == 'Municipal') & (df_especial['Estadual'] == 'Estadual'), 'Encerramento'] = 'Municipal e Estadual'

df_especial['Encerramento'].unique()

array(['Nao encerrado', 'Estadual', 'Municipal', 'Municipal e Estadual',
       'Federal'], dtype=object)

In [10]:
# pesquisa por email encerramentos incorretos
df_especial.loc[df_especial['Encerramento Municipal - E-mail Responsável'].str.contains('carolina-krieger@saude.rs.gov.br', na = False), 'Encerramento'] = 'Estadual'
df_especial.loc[df_especial['Encerramento Municipal - E-mail Responsável'].str.contains('helena-ramos@saude.rs.gov.br', na = False), 'Encerramento'] = 'Estadual'
df_especial.loc[df_especial['Encerramento Municipal - E-mail Responsável'].str.contains('rosemeri-cohen@saude.rs.gov.br', na = False), 'Encerramento'] = 'Estadual'
df_especial.loc[df_especial['Encerramento Municipal - E-mail Responsável'].str.contains('aczanchi@gmail.com', na = False), 'Encerramento'] = 'Estadual'

In [15]:
# analisar manualmente os emails de quem encerrou casos municipais
teste = df_especial.copy()
teste = teste[['Encerramento Municipal - E-mail Responsável']]
teste = teste.groupby(by=['Encerramento Municipal - E-mail Responsável']).count()
teste.to_excel('E-mail encerrados.xlsx')

VACINAS

In [98]:
df_especial['Vacina'] = pd.Series()

pfizer = ['pfizer', 'comirnaty']
astra = ['astrazeneca', 'covishield', 'oxford', 'fiocruz']
butantan = ['butantan', 'coronavac', 'sinovac']

df_especial.loc[df_especial['Imunobiológico (vacina)'].str.contains('janssen'), 'Vacina'] = 'Janssen'
df_especial.loc[df_especial['Imunobiológico (vacina)'].str.contains('(?i)' + '|'.join(pfizer)), 'Vacina'] = 'Pfizer'
df_especial.loc[df_especial['Imunobiológico (vacina)'].str.contains('(?i)' + '|'.join(astra)), 'Vacina'] = 'Astrazeneca'
df_especial.loc[df_especial['Imunobiológico (vacina)'].str.contains('(?i)' + '|'.join(butantan)), 'Vacina'] = 'Butantan'

# GERAR EXCEL

In [103]:
# filtrar colunas de interesse
df_especial = df_especial[['Número da Notificação', 'Sexo', 'Vacina',    # Informações extras
                          'Idade Evento', 'Faixa etaria',                # Gráfico por faixa etária
                          'SE',                                          # Gráfico por semana epidemiologica
                          'Tipo de Evento',                              # Gráfico por tipo de evento
                          'Encerramento', 'Município da Notificação',    # Gráfico encerrado por municipio
                          'Gravidade']]                                  # Gráfico por gravidade                         

In [6]:
df_especial.to_excel('Banco_dados_completo.xlsx')